In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#URLs with corresponding years
urls = [
    ('https://www.pro-football-reference.com/draft/2024-combine.htm', 2024),
    ('https://www.pro-football-reference.com/draft/2023-combine.htm', 2023),
    ('https://www.pro-football-reference.com/draft/2022-combine.htm', 2022),
    ('https://www.pro-football-reference.com/draft/2021-combine.htm', 2021),
    ('https://www.pro-football-reference.com/draft/2020-combine.htm', 2020),
    ('https://www.pro-football-reference.com/draft/2019-combine.htm', 2019),
    ('https://www.pro-football-reference.com/draft/2018-combine.htm', 2018),
    ('https://www.pro-football-reference.com/draft/2017-combine.htm', 2017),
    ('https://www.pro-football-reference.com/draft/2016-combine.htm', 2016),
    ('https://www.pro-football-reference.com/draft/2015-combine.htm', 2015),
    ('https://www.pro-football-reference.com/draft/2014-combine.htm', 2014)
]


In [10]:
# intitialize to store the dataframes
dfs = []

# big loop for each URL
for url, year in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    # table containing the combine data
    table = soup.find('table', {'id': 'combine'})
    
    #column headers
    headers = [th.text for th in table.find('thead').find_all('th')]
    
    # Extract data rows
    data = []
    for row in table.find('tbody').find_all('tr'):
        row_data = [td.text for td in row.find_all('td')]
        # Include player name from the first column
        player_name = row.find('th').text
        row_data.insert(0, player_name)
        # Adjust the number of columns to match the number of headers
        if len(row_data) == len(headers) - 1:
            row_data.insert(0, '')  # Insert an empty string for the missing column
        data.append(row_data)
    
    # Create DataFrame
    df = pd.DataFrame(data, columns=headers)
    
    # Check if the 'Drafted (tm/rnd/yr)' column exists before dropping it
    if 'Drafted (tm/rnd/yr)' in df.columns:
        df.drop(columns=['Drafted (tm/rnd/yr)'], inplace=True)
    
    # Add a column for the year
    df['Year'] = year
    dfs.append(df)


ten_yr = pd.concat(dfs, ignore_index=True)
# display(ten_yr)

In [11]:
positions = ['QB', 'RB', 'WR', 'TE', 'OT', 'G', 'C', 'EDGE', 'DT', 'DE', 'LB', 'CB', 'DB', 'K', 'P']

#initialize dictionary
filtered_dfs = {}

# run over each position
for position in positions:
    if position == 'G':
        filtered_dfs[position] = ten_yr[ten_yr['Pos'].isin(['G', 'OG', 'OL'])].copy()
    elif position == 'DB':
        filtered_dfs[position] = ten_yr[ten_yr['Pos'].isin(['SAF', 'DB', 'S'])].copy()
    elif position == 'LB':
        filtered_dfs[position] = ten_yr[ten_yr['Pos'].isin(['LB', 'ILB', 'OLB'])].copy()
    elif position =='RB':
        filtered_dfs[position] = ten_yr[ten_yr['Pos'].isin(['RB', 'FB'])]
    elif position == 'C':
        filtered_dfs[position] = ten_yr[ten_yr['Pos'].isin(['C', 'LS'])]
    else:
        filtered_dfs[position] = ten_yr[ten_yr['Pos'] == position].copy()

# Access filtered DataFrames by position
qb_df = filtered_dfs['QB']
rb_df = filtered_dfs['RB']
wr_df = filtered_dfs['WR']
te_df = filtered_dfs['TE']
ot_df = filtered_dfs['OT']
g_df = filtered_dfs['G']
c_df = filtered_dfs['C']
edge_df = filtered_dfs['EDGE']
dt_df = filtered_dfs['DT']
de_df = filtered_dfs['DE']
lb_df = filtered_dfs['LB']
cb_df = filtered_dfs['CB']
db_df = filtered_dfs['DB']
k_df = filtered_dfs['K']
p_df = filtered_dfs['P']

display(qb_df)

,Player,Pos,School,College,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle,Year
57,Jayden Daniels,QB,LSU,College Stats,6-4,210,,,,,,,2024
114,Sam Hartman,QB,Notre Dame,College Stats,6-1,211,4.80,28.5,,109,7.19,4.34,2024
151,Michael Penix Jr.,QB,Washington,College Stats,6-2,216,,,,,,,2024
170,Devin Leary,QB,Kentucky,College Stats,6-1,215,,,,,,,2024
182,Drake Maye,QB,North Carolina,College Stats,6-4,223,,,,,,,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3751,Tom Savage,QB,Pittsburgh,College Stats,6-4,228,4.97,27.0,,105,7.33,4.36,2014
3755,Connor Shaw,QB,South Carolina,College Stats,6-0,206,4.66,34.0,,116,7.07,4.33,2014
3784,Logan Thomas,QB,Virginia Tech,College Stats,6-6,248,4.61,35.5,,118,7.05,4.18,2014
3797,Dustin Vaughan,QB,West Texas A&M,,6-5,235,4.95,29.0,,108,7.25,4.43,2014


## Keeping this code because it's useful when debugging. File being run elsewhere

In [12]:
counts = {}

# Iterate over each position and count the number of rows in the corresponding DataFrame
total_count = 0
for position in positions:
    count = filtered_dfs[position].shape[0]
    counts[position] = count
    total_count += count

# Display the counts
for position, count in counts.items():
    print(f"Number of players in {position}: {count}")

# Display the total count
print(f"Total number of players: {total_count}")

Number of players in QB: 181
Number of players in RB: 357
Number of players in WR: 532
Number of players in TE: 218
Number of players in OT: 236
Number of players in G: 292
Number of players in C: 102
Number of players in EDGE: 127
Number of players in DT: 205
Number of players in DE: 168
Number of players in LB: 390
Number of players in CB: 393
Number of players in DB: 319
Number of players in K: 52
Number of players in P: 59
Total number of players: 3631


In [20]:
# Define a function to calculate sum for specific columns in a DataFrame
def calculate_sum(df, columns):
    # Initialize an empty DataFrame to store sums
    sum_df = pd.DataFrame()

    # Iterate over each column specified in the 'columns' list
    for column in columns:
        # Convert non-numeric values to NaN
        df[column] = pd.to_numeric(df[column], errors='coerce')
        # Sum the values in the column
        column_sum = df[column].sum()
        # Add sum to the sum DataFrame
        sum_df[column + '_sum'] = [column_sum]

    return sum_df

# Define the columns for which you want to calculate sum
sum_columns = ['40yd']

qb_avg = qb_sum_df['40yd_sum'] / len(qb_df)
rb_avg = rb_sum_df['40yd_sum'] / len(rb_df)
wr_avg = wr_sum_df['40yd_sum'] / len(wr_df)
te_avg = te_sum_df['40yd_sum'] / len(te_df)
ot_avg = ot_sum_df['40yd_sum'] / len(ot_df)
g_avg = g_sum_df['40yd_sum'] / len(g_df)
c_avg = c_sum_df['40yd_sum'] / len(c_df)
edge_avg = edge_sum_df['40yd_sum'] / len(edge_df)
dt_avg = dt_sum_df['40yd_sum'] / len(dt_df)
de_avg = de_sum_df['40yd_sum'] / len(de_df)
lb_avg = lb_sum_df['40yd_sum'] / len(lb_df)
cb_avg = cb_sum_df['40yd_sum'] / len(cb_df)
db_avg = db_sum_df['40yd_sum'] / len(db_df)
k_avg = k_sum_df['40yd_sum'] / len(k_df)
p_avg = p_sum_df['40yd_sum'] / len(p_df)

print("Average 40yd for QB:", qb_avg)
print("Average 40yd for RB:", rb_avg)
print("Average 40yd for WR:", wr_avg)
print("Average 40yd for TE:", te_avg)
print("Average 40yd for OT:", ot_avg)
print("Average 40yd for G:", g_avg)
print("Average 40yd for C:", c_avg)
print("Average 40yd for EDGE:", edge_avg)
print("Average 40yd for DT:", dt_avg)
print("Average 40yd for DE:", de_avg)
print("Average 40yd for LB:", lb_avg)
print("Average 40yd for CB:", cb_avg)
print("Average 40yd for DB:", db_avg)
print("Average 40yd for K:", k_avg)
print("Average 40yd for P:", p_avg)

Average 40yd for QB: 0    3.686796
Name: 40yd_sum, dtype: float64
Average 40yd for RB: 0    3.982017
Name: 40yd_sum, dtype: float64
Average 40yd for WR: 0    3.97344
Name: 40yd_sum, dtype: float64
Average 40yd for TE: 0    3.527936
Name: 40yd_sum, dtype: float64
Average 40yd for OT: 0    4.298008
Name: 40yd_sum, dtype: float64
Average 40yd for G: 0    4.443151
Name: 40yd_sum, dtype: float64
Average 40yd for C: 0    4.121176
Name: 40yd_sum, dtype: float64
Average 40yd for EDGE: 0    3.306614
Name: 40yd_sum, dtype: float64
Average 40yd for DT: 0    4.382585
Name: 40yd_sum, dtype: float64
Average 40yd for DE: 0    4.334524
Name: 40yd_sum, dtype: float64
Average 40yd for LB: 0    3.831667
Name: 40yd_sum, dtype: float64
Average 40yd for CB: 0    3.952595
Name: 40yd_sum, dtype: float64
Average 40yd for DB: 0    3.77442
Name: 40yd_sum, dtype: float64
Average 40yd for K: 0    1.608077
Name: 40yd_sum, dtype: float64
Average 40yd for P: 0    2.826271
Name: 40yd_sum, dtype: float64
